## Input

Input files:

In [1]:
# input data
outdir     = "results_hap_analysis/"
metasam_fn = "metadata/samples.meta_phenotypes.txt"
callset_fn = "/media/xavi/Saigon/Variation/phase2.AR1_ALL/variation/ag1000g.phase2.ar1.2R.h5"
snpeff_fn  = "/media/xavi/Saigon/Variation/phase2.AR1_ALL/snpeff/ag1000g.phase2.ar1.snpeff.AgamP4.2.2R.h5"
accessi_fn = "/home/xavi/Documents/VariationAg1k/data/phase2.AR1/accessibility/accessibility.h5"
gffann_fn  = "metadata/Anopheles-gambiae-PEST_BASEFEATURES_AgamP4.9.gff3"

In [2]:
import numpy as np
import pandas as pd
import allel
import h5py
import scipy
from scipy.spatial.distance import squareform
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns

In [3]:
# define populations
outcode    = "out"
popl       = ["AOcol","BFcol","BFgam","CIcol","CMgam","FRgam","GAgam","GHcol","GHgam","GM","GNcol","GNgam","GQgam","GW","KE","UGgam"]
popc       = "population"

# gene coordinates
chrom = "2R"
ace_start = 3489213
ace_end   = 3493788
ace_119S  = 3492074

# duplication coordinates
ace_dups  = 3436800 # start duplication
ace_dupe  = 3639600 # end duplication

# exclude these samples
excludec   = "ox_code"
excludel   = ["NO RES"]
# traits to subset
sub1c      = "population"
sub1l      = ["AOcol","BFcol","BFgam","CIcol","CMgam","FRgam","GAgam","GHcol","GHgam","GM","GNcol","GNgam","GQgam","GW","KE","UGgam"]
# min frq to retain minor allele
minfrq     = 0.05

# flanking bp or num haps, to retain in analysis
fbp_var    = 0     # num bp to retain around loci of interest (gene), should be 0

In [4]:
# load plot settings
sns.set(context="notebook",style="ticks",
        font_scale=1,font="Arial",palette="bright")

## Load data

Load genotypes, haplotypes, variants, effects, etc:

In [5]:
# LOAD DATA
print("Samples bool...")
# load samples list with sample code, groupings, locations etc.
samples_df   = pd.read_csv(metasam_fn, sep='\t')
samples_bool = (
    samples_df[popc].isin(popl).values & 
    samples_df[sub1c].isin(sub1l).values &
    ~samples_df[excludec].isin(excludel).values)
samples_sub  = samples_df[samples_bool]
samples_sub.reset_index(drop=True, inplace=True)


# genotypes: variants
# callset     = zarr.open(callset_fn)
callset     = h5py.File(callset_fn,mode="r")
snpeff_zr   = h5py.File(snpeff_fn,mode="r")
print("Load variants...")
callset_var = callset[chrom]["variants"]
genvars     = allel.VariantChunkedTable(callset_var,names=["POS","REF","ALT","num_alleles"],index="POS") # variants
genveff     = allel.VariantChunkedTable(snpeff_zr[chrom]["variants"],names=["POS","ANN"],index="POS") # variant effects
pos_bool    = np.logical_and(genvars["POS"] >= ace_dups - fbp_var, 
                             genvars["POS"] <= ace_dupe + fbp_var)
genvars_sub = genvars.compress(pos_bool)
genveff_sub = genveff[:].compress(pos_bool)

# genotypes: calldata
print("Load genotypes...")
callset_gen = callset[chrom]["calldata"]["genotype"]
genotyp     = allel.GenotypeChunkedArray(callset_gen)
genotyp_sub = genotyp.subset(sel0=pos_bool, sel1=samples_bool)

Samples bool...
Load variants...
Load genotypes...


In [6]:
print("Samples dictionary...")
# indexed dictionary of populations
popdict = dict()
for popi in popl: 
    popdict[popi]  = samples_sub[samples_sub[popc] == popi].index.tolist()

# add an extra population composed of all other locations
popdict["all"] = []
for popi in popl:
    popdict["all"] = popdict["all"] + popdict[popi]

gen_pops_count = samples_sub.groupby(popc).size()
gen_pops_count

Samples dictionary...


population
AOcol     78
BFcol     75
BFgam     92
CIcol     71
CMgam    297
FRgam     24
GAgam     69
GHcol     55
GHgam     12
GM        65
GNcol      4
GNgam     40
GQgam      9
GW        91
KE        48
UGgam    112
dtype: int64

In [7]:
# genotypes: allele counts
print("Allele counts genotypes...")
genalco_sub = genotyp_sub.count_alleles_subpops(subpops=popdict)

Allele counts genotypes...


In [8]:
# SUBSET DATA

# filter genotypes: segregating alleles, biallelic, no singletons, coding
is_seg      = genalco_sub["all"].is_segregating()[:]   # segregating in at least one population?
is_bia      = genvars_sub["num_alleles"] == 2          # biallelic "==2" | multiallelic ">1"
is_nosing   = genalco_sub["all"][:,:2].min(axis=1) > 1 # no singletons
is_coding   = np.asarray([genveff_sub["ANN"][i][1].astype(str) == "missense_variant" for i,_ in enumerate(genveff_sub["ANN"])])

# subset data: keep variants with at least 5% freq in at least one pop
co_major = pd.DataFrame() # data={"chr" : chrom, "pos" : genvars_sub["POS"][:]}
co_minor = pd.DataFrame()
fq_minor = pd.DataFrame()
for popi in popl: 
    co_major[popi] = genalco_sub[popi][:,0]
    co_minor[popi] = genalco_sub[popi][:,1]
    fq_minor[popi] = genalco_sub[popi][:,1] / genalco_sub[popi][:,0:2].sum(axis=1)
co_major["all"] = co_major.sum(axis=1)
co_minor["all"] = co_minor.sum(axis=1)
fq_minor["all"] = co_minor.sum(axis=1) / (co_minor.sum(axis=1)+co_major.sum(axis=1))
is_minfq = fq_minor.max(axis=1) > minfrq
# fqminb1 = fq_minor.max(axis=1) > minfrq
# fqminb2 = fq_minor.min(axis=1) < 1-minfrq
# is_minfq = np.logical_and(fqminb1,fqminb2)
#is_minfq = fq_minor["all"] > 0.05

# which variants to keep
filsnp_bool = (is_seg[:] & is_nosing[:] & is_minfq[:])

# report
print("loci: %s:%i-%i\n%i filt vars\n%i vars in region, %i in chr" % 
      (chrom,ace_dups,ace_dupe,np.sum(filsnp_bool),len(genvars_sub),len(genvars)))

loci: 2R:3436800-3639600
22805 filt vars
107810 vars in region, 24767689 in chr


/home/xavi/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


Now, apply filters:

In [9]:
# apply filters
print("Apply filters...")
genotyp_seg = genotyp_sub.compress(filsnp_bool)
genvars_seg = genvars_sub.compress(filsnp_bool)
genalco_seg = genalco_sub.compress(filsnp_bool)
genveff_seg = genveff_sub[:].compress(filsnp_bool)
    
# SNP name
snpname_seg = np.array(
    [chrom+":"+x1+" "+x2+" "+x3 for x1,x2,x3 in zip(
        np.asarray(genvars_seg["POS"]).astype(str),
        np.asarray([genveff_seg["ANN"][i][10].astype(str) for i,_ in enumerate(genveff_seg["ANN"])]),
        np.asarray([genveff_seg["ANN"][i][3].astype(str) for i,_ in enumerate(genveff_seg["ANN"])])
    )]
)

Apply filters...


Find variant where the `G119S` variant is (actually `G280S`):

In [10]:
intvar_pos = 3492074
intvar_ix  = np.where(genvars_seg[:]["POS"]==intvar_pos)[0][0]

print("pos",intvar_pos,"index",intvar_ix)

pos 3492074 index 6799


In [11]:
for popi in popl:
    print(
        popi,"\t",
        genalco_seg[popi][int(intvar_ix)][0],"\t",
        genalco_seg[popi][int(intvar_ix)][1],"\t",
        genalco_seg[popi][int(intvar_ix)][1]/sum(genalco_seg[popi][int(intvar_ix)])
    )

AOcol 	 156 	 0 	 0.0
BFcol 	 147 	 3 	 0.02
BFgam 	 155 	 29 	 0.15760869565217392
CIcol 	 80 	 62 	 0.43661971830985913
CMgam 	 594 	 0 	 0.0
FRgam 	 48 	 0 	 0.0
GAgam 	 138 	 0 	 0.0
GHcol 	 103 	 7 	 0.06363636363636363
GHgam 	 8 	 16 	 0.6666666666666666
GM 	 130 	 0 	 0.0
GNcol 	 8 	 0 	 0.0
GNgam 	 71 	 9 	 0.1125
GQgam 	 18 	 0 	 0.0
GW 	 182 	 0 	 0.0
KE 	 96 	 0 	 0.0
UGgam 	 224 	 0 	 0.0


Output table with genotypes per sample in the G119S variant:

In [12]:
genotyp_seg[:].to_n_alt(fill=-1)[intvar_ix]

array([0, 0, 0, ..., 1, 1, 1], dtype=int8)

In [13]:
num_popa = co_minor.shape[1]
df = pd.DataFrame(data={
    "ox_code"    : samples_sub["ox_code"].values,
    "population" : samples_sub["population"].values,
    "genotype"   : genotyp_seg[:].to_n_alt(fill=-1)[intvar_ix],
    "phenotype"  : samples_sub["phenotype"].values,
},
columns=["ox_code","population","genotype","phenotype"]
)
df.to_csv("%s/Genotype_3492074_G119S.csv" % (outdir),sep="\t",index=False)

Do the same thing for A65S, which is also segregating in CI:

In [14]:
jntvar_pos = 3489405
jntvar_ix  = np.where(genvars_seg[:]["POS"]==jntvar_pos)[0][0]

print("pos",jntvar_pos,"index",jntvar_ix)

pos 3489405 index 6592


In [15]:
genotyp_seg[:].to_n_alt(fill=-1)[jntvar_ix]

array([1, 1, 2, ..., 1, 1, 2], dtype=int8)

In [16]:
num_popa = co_minor.shape[1]
df = pd.DataFrame(data={
    "ox_code"    : samples_sub["ox_code"].values,
    "population" : samples_sub["population"].values,
    "genotype"   : genotyp_seg[:].to_n_alt(fill=-1)[jntvar_ix],
    "phenotype"  : samples_sub["phenotype"].values,
},
columns=["ox_code","population","genotype","phenotype"]
)
df.to_csv("%s/Genotype_3489405_A65S.csv" % (outdir),sep="\t",index=False)

## Linkage disequilibrium

Calculate linkage disequilibrium for all variants within the duplication:

In [17]:
# linkage disequilibrium
print("LD Rogers & Huff...")
ld_rhr = allel.rogers_huff_r(genotyp_seg.to_n_alt(fill=-1))
ld_rhr = squareform(ld_rhr)
np.fill_diagonal(ld_rhr,np.nan)

LD Rogers & Huff...


In [18]:
# linkage disequilibrium
print("LD Rogers & Huff...")
ld_rhr_CIcol = allel.rogers_huff_r(genotyp_seg.subset(sel1=popdict["CIcol"]).to_n_alt(fill=-1))
ld_rhr_CIcol = squareform(ld_rhr_CIcol)
np.fill_diagonal(ld_rhr_CIcol,np.nan)

LD Rogers & Huff...


Print table:

In [19]:
# report
# variant effects (variants in region of interest: duplication)
fq_repor = pd.DataFrame(data={
    "chr" : chrom,
    "POS" : genvars_seg["POS"][:].astype(str),
    "REF" : genvars_seg["REF"][:].astype(str),
    "ALT" : genvars_seg["ALT"][:].astype(str)[:,0],
    "gene_eff" : [genveff_seg["ANN"][i][3].astype(str)  for i,_ in enumerate(genveff_seg["ANN"])],
    "PEP_eff"  : [genveff_seg["ANN"][i][10].astype(str) for i,_ in enumerate(genveff_seg["ANN"])],
    "CDS_eff"  : [genveff_seg["ANN"][i][9].astype(str)  for i,_ in enumerate(genveff_seg["ANN"])],
},
)

# add global LD to 280S
fq_repor["LDr_allpops"] = ld_rhr[intvar_ix,]
fq_repor["LDr_CIcol"]   = ld_rhr_CIcol[intvar_ix,]

# add minor and major counts and frequencies
fq_repos = pd.concat( [ fq_minor[filsnp_bool], co_minor[filsnp_bool] ], axis=1)
fq_repos = pd.concat( [ fq_repos,              co_major[filsnp_bool] ], axis=1)
fq_repos_newcols = [str(col) + '_fqmin' for col in fq_repos.columns[0:num_popa]] + [str(col) + '_comin' for col in fq_repos.columns[num_popa:num_popa*2]] + [str(col) + '_comaj' for col in fq_repos.columns[num_popa*2:num_popa*3]]
fq_repos.columns = fq_repos_newcols
fq_repor = pd.concat([fq_repor,fq_repos.reset_index(drop=True)], axis=1)

# print table
fq_repor.to_csv("%s/AlleleFq_tab.csv" % (outdir),sep="\t",index=False)

Find which variants in the duplication have high LD (`LD>0.95`) with variant `G119S`:

In [20]:
ld_max    = max(ld_rhr[intvar_ix])
ld_max_ix = np.where(ld_rhr[intvar_ix] > 0.95)

genilist  = []
geniname  = []
for ixn,ixi in enumerate(ld_max_ix[0]):
    genilist.append(genvars_seg["POS"][ixi])
    geniname.append(snpname_seg[ixi])
    
    
    print(snpname_seg[ixi],
          "\tindex =",ld_max_ix[0][ixn],
          "\tLD =",ld_rhr[intvar_ix][ixi],
          "\tfq CIcol = ",np.asarray(fq_minor["CIcol"].compress(filsnp_bool))[ixi]
         )


2R:3444618 . AGAP001355-AGA 	index = 901 	LD = 0.97264516 	fq CIcol =  0.43661971830985913
2R:3445910 . AGAP001355-AGA 	index = 1055 	LD = 0.9504605 	fq CIcol =  0.43661971830985913
2R:3447556 . AGAP001355-AGA 	index = 1234 	LD = 0.96361303 	fq CIcol =  0.43661971830985913
2R:3456166 . AGAP001355-AGA 	index = 2415 	LD = 0.9675373 	fq CIcol =  0.43661971830985913
2R:3456937 . AGAP001355-AGA 	index = 2507 	LD = 0.97637683 	fq CIcol =  0.43661971830985913
2R:3459582 . AGAP001355-AGA 	index = 2840 	LD = 0.9591772 	fq CIcol =  0.43661971830985913
2R:3464705 . AGAP001355-AGA 	index = 3411 	LD = 0.96810204 	fq CIcol =  0.43661971830985913
2R:3465693 . AGAP001355-AGA 	index = 3670 	LD = 0.96810204 	fq CIcol =  0.43661971830985913
2R:3466268 . AGAP001355-AGA 	index = 3733 	LD = 0.9719312 	fq CIcol =  0.43661971830985913
2R:3469441 . AGAP001355-AGA 	index = 4121 	LD = 0.95479333 	fq CIcol =  0.43661971830985913
2R:3469672 . AGAP001355-AGA 	index = 4151 	LD = 0.96308875 	fq CIcol =  0.43661971830

/home/xavi/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in greater
  
/home/xavi/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: Series.compress(condition) is deprecated. Use 'Series[condition]' or 'np.asarray(series).compress(condition)' instead.
  
